In [ ]:
from pathlib import Path

import cv2
import numpy as np
import os

from ultralytics import YOLO

# https://docs.ultralytics.com/guides/isolating-segmentation-objects/#recipe-walk-through 
# https://docs.ultralytics.com/modes/predict/#key-features-of-predict-mode 

m = YOLO("yolo11n-seg.pt")  
directory = './Images'
# some path to images
files = [directory + f.split('/')[-1] for f in os.listdir(directory)]
res = m([files])  

# Iterate detection results 
for r in res:
    img = np.copy(r.orig_img)
    img_name = Path(r.path).stem

     # Iterate each object contour
    for ci, c in enumerate(r):
        label = c.names[c.boxes.cls.tolist().pop()]

        # Create an empty mask
        b_mask = np.zeros(img.shape[:2], np.uint8)

        # Draw the contour of the object on the mask (object as white, background as black)
        contour = c.masks.xy.pop().astype(np.int32).reshape(-1, 1, 2)
        _ = cv2.drawContours(b_mask, [contour], -1, 255, cv2.FILLED)

        # Save the mask as a binary mask file (white object, black background)
        _ = cv2.imwrite(f"./Masks/{img_name}.png", b_mask)

        # # OPTION: Isolate object with transparent background (useful for PNGs)
        # isolated = np.dstack([img, b_mask])

        # # Crop image to the object region using bounding box coordinates
        # x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32)
        # iso_crop = isolated[y1:y2, x1:x2]

        # # Save the isolated object with transparency
        # _ = cv2.imwrite("./Masks/{img_name}_{label}-{ci}.png", iso_crop)